In [1]:
clean_up = True
%run stdPackages.ipynb
ws = gams.GamsWorkspace(working_directory=d['work']) # specify where you want to run the GAMS models from (here the repository referred to in d['work'])
d['data'] = os.path.join(d['data'], 'IO2018')

The file _gams_py_gdb0.gdx is still active and was not deleted.
The file _gams_py_gdb1.gdx is still active and was not deleted.


## GR18 Production, debug

Different attempts at diagnosing problems with the production module/data.

### Data/settings

In [2]:
name = 'smallGR' # name of model
module = name+'_P' # name of module
db_IO = GpyDB(pickle_path = os.path.join(d['data'], f'IO_{name}')) # load IO database named IO_name
with open(f"{d['data']}\\glob_{name}","rb") as file: # load global settings anmed glob_name
    glob=pickle.load(file)
db_m  = GpyDB(pickle_path = os.path.join(d['data'], 'db_'+module)) # load data with settings for production module

Add time index to the IO data:

In [3]:
def addT(symbol, t):
    return adjMultiIndex.bc(symbol, t).reorder_levels(['t']+symbol.index.names if 't' not in symbol.index.names else symbol.index.names)
[db_IO.__setitem__(k, addT(db_IO.get(k), glob.db['t'].vals)) for k in db_IO.getTypes(['variable','scalar_variable'])];

Add durable prices (for now, to solve static calibration):

In [4]:
db_IO['pD'] = db_IO.get('pD_dur').combine_first(db_IO.get('pD'))
db_IO['p'] = db_IO.get('pD_dur').groupby(['t','n']).mean().combine_first(db_IO.get('p'))

### Sector-by-sector solution

Sector *other1:*

In [5]:
sector = pd.Index(['other1'],name='s')

In [6]:
nest = adj.rc_pd(db_m['nestProduction'], sector)
db_s = db_IO.copy()
aggregateDB.subset_db(db_s, sector)
tree = nestingTree.tree(name = 'CESnest', tree = nest.to_list()) # individual tree
Tree = nestingTree.aggTree(name = 'stdProduction', trees = {tree.name: tree})(namespace = {n+'_input': n for n in db_s.get('n')})
P = CGE_Production.Production(tree=Tree, glob = glob) # initialize module from nesting tree and global settings
aggregateDB.subset_db(db_s, Tree.get('n')) # goes through all symbols in db_IO and only keep the elements that are in the set 'n' from Tree.db
robust.robust_merge_dbs(P.s.db, db_s, priority = 'second') # Merge IO data into the database of the module; if a symbol is in both, prioritize records from the second database

Try to run one sector at a time:

In [5]:
for s in db_IO.get('s_p'):
    sector = pd.Index([s], name = 's')
    # subset data:
    nest = adj.rc_pd(db_m['nestProduction'], sector)
    db_s = db_IO.copy()
    aggregateDB.subset_db(db_s, sector)
    tree = nestingTree.tree(name = 'CESnest', tree = nest.to_list()) # individual tree
    Tree = nestingTree.aggTree(name = 'stdProduction', trees = {tree.name: tree})(namespace = {n+'_input': n for n in db_s.get('n')})
    P = CGE_Production.Production(tree=Tree, glob = glob) # initialize module from nesting tree and global settings
    aggregateDB.subset_db(db_s, Tree.get('n')) # goes through all symbols in db_IO and only keep the elements that are in the set 'n' from Tree.db
    robust.robust_merge_dbs(P.s.db, db_s, priority = 'second') # Merge IO data into the database of the module; if a symbol is in both, prioritize records from the second database
    v = valueShares.valueShares(Tree, db_s)
    v.compile() # set up model structure, and make sure to initialize symbols if they are not yet defined in the database 
    v.write(); # write GAMS code used for the model
    m = v.run(exportTo = d['work'],ws=ws) # solve the "model".
    m.out_db.get('vD').index = m.out_db.get('vD').index.set_levels(m.out_db.get('vD').index.levels[1].astype(str), level = 's')
    m.out_db.get('mu').index = m.out_db.get('mu').index.set_levels(m.out_db.get('mu').index.levels[1].astype(str), level = 's')
    gpyDB.GpyDBs_AOM_Second(P.s.db, gpy(adj.rc_pd(m.out_db.get('vD'), P.get('int')).rename('qD'))) # set intermediate goods levels
    gpyDB.GpyDBs_AOM_Second(P.s.db, gpy(m.out_db.get('mu').xs(P.get('t0')[0]).rename('mu'))) # set intermediate goods levels
    P.s.db['tauS'] = adjMultiIndex.applyMult(db_s.get('TotalTax'), P.get('output')) / P.get('qS')
    P.compile(initDB=True) # set up model structure, and make sure to initialize symbols if they are not yet defined in the database (initDB = True)
    P.s.setstate('C') # set to calibration state
    P.write(); # write GAMS code
    mStatic = P.run(exportTo = d['work'],ws=ws,**{'cns': 'CONOPT4'}) # solve the model using CONOPT4.